In [1]:
!git clone https://github.com/shariqfarooq123/AdaBins.git

Cloning into 'AdaBins'...
remote: Enumerating objects: 80, done.
remote: Total 80 (delta 0), reused 0 (delta 0), pack-reused 80
Unpacking objects: 100% (80/80), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/AdaBins

/content/AdaBins


In [4]:
!git clone https://github.com/dontLoveBugs/DORN_pytorch.git

Cloning into 'DORN_pytorch'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 254 (delta 17), reused 41 (delta 14), pack-reused 203
Receiving objects: 100% (254/254), 514.28 KiB | 11.18 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [ ]:
!pip install pytorch3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 30.0 MB 89.0 MB/s 
     |████████████████████████████████| 50 kB 8.3 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=f203a1ae75765d8be91a10b4a830386e25e1a133f1d44119035a4e719643f855
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built fvcore
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 8.0 MB/s 
     |████████████████████████████████| 145 kB 45.8 MB/s 
     |████████████████████████████████| 181 kB 71.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=343ede394d6a9c02865a34e2ca3e3c16f8a93479e2631d92a401171d24e0e11e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install pyyaml==5.1 pycocotools>=2.0.1
!pip install -U fvcore
!pip install imgaug==0.2.6
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
import os
!curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
!tar xzf 1.10.0.tar.gz
os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
!pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 22 kB/s 
     |████████████████████████████████| 6.6 MB 25.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.5.0+cu101 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.5.0+cu101

In [ ]:
!wandb login

In [ ]:
!wget http://sceneparsing.csail.mit.edu/model/pretrained_resnet/resnet101-imagenet.pth

--2022-03-28 16:45:57--  http://sceneparsing.csail.mit.edu/model/pretrained_resnet/resnet101-imagenet.pth
Resolving sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)... 128.30.195.26
Connecting to sceneparsing.csail.mit.edu (sceneparsing.csail.mit.edu)|128.30.195.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179197864 (171M)
Saving to: ‘resnet101-imagenet.pth’

resnet101-imagenet. 100%[===================>] 170.90M  67.7MB/s    in 2.5s    

2022-03-28 16:46:00 (67.7 MB/s) - ‘resnet101-imagenet.pth’ saved [179197864/179197864]



In [ ]:
#download depth
import os
import sys
import h5py
import torch
import shutil
import random
import tarfile
import zipfile
import numpy as np

from PIL import Image
from torch.utils.data import Dataset
from torchvision.datasets.utils import download_url

def _create_depth_files(mat_file: str, root: str, train_ids: list):
    """
    Extract the depth arrays from the mat file into images
    :param mat_file: path to the official labelled dataset .mat file
    :param root: The root directory of the dataset
    :param train_ids: the IDs of the training images as string (for splitting)
    """
    os.mkdir(os.path.join(root, "train_depth"))
    os.mkdir(os.path.join(root, "test_depth"))
    train_ids = set(train_ids)

    depths = h5py.File(mat_file, "r")["depths"]
    for i in range(len(depths)):
        img = (depths[i] * 1e4).astype(np.uint16).T
        id_ = str(i + 1).zfill(4)
        folder = "train" if id_ in train_ids else "test"
        save_path = os.path.join(root, f"{folder}_depth", id_ + ".png")
        Image.fromarray(img).save(save_path)

def download_depth(root: str):
    url = (
        "http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat"
    )
    train_dst = os.path.join(root, "train_depth")
    test_dst = os.path.join(root, "test_depth")

    if not os.path.exists(train_dst) or not os.path.exists(test_dst):
        tar = os.path.join(root, url.split("/")[-1])
        if not os.path.exists(tar):
            download_url(url, root)
        if os.path.exists(tar):
            train_ids = [
                f.split(".")[0] for f in os.listdir(os.path.join(root, "train_rgb"))
            ]
            _create_depth_files(tar, root, train_ids)
download_depth("/content/drive/MyDrive/data")

0it [00:00, ?it/s]

In [ ]:
def _unpack(file: str):
    """
    Unpacks tar and zip, does nothing for any other type
    :param file: path of file
    """
    path = file.rsplit(".", 1)[0]

    if file.endswith(".tgz"):
        tar = tarfile.open(file, "r:gz")
        tar.extractall(path)
        tar.close()
    elif file.endswith(".zip"):
        zip = zipfile.ZipFile(file, "r")
        zip.extractall(path)
        zip.close()
def _rename_files(folder: str, rename_func: callable):
    """
    Renames all files inside a folder based on the passed rename function
    :param folder: path to folder that contains files
    :param rename_func: function renaming filename (not including path) str -> str
    """
    imgs_old = os.listdir(folder)
    imgs_new = [rename_func(file) for file in imgs_old]
    for img_old, img_new in zip(imgs_old, imgs_new):
        shutil.move(os.path.join(folder, img_old), os.path.join(folder, img_new))


def _replace_folder(src: str, dst: str):
    """
    Rename src into dst, replacing/overwriting dst if it exists.
    """
    if os.path.exists(dst):
        shutil.rmtree(dst)
    shutil.move(src, dst)


def download_rgb(root: str):
    train_url = "http://www.doc.ic.ac.uk/~ahanda/nyu_train_rgb.tgz"
    test_url = "http://www.doc.ic.ac.uk/~ahanda/nyu_test_rgb.tgz"

    def _proc(url: str, dst: str):
        if not os.path.exists(dst):
            tar = os.path.join(root, url.split("/")[-1])
            if not os.path.exists(tar):
                download_url(url, root)
            if os.path.exists(tar):
                _unpack(tar)
                _replace_folder(tar.rstrip(".tgz"), dst)
                _rename_files(dst, lambda x: x.split("_")[2])

    _proc(train_url, os.path.join(root, "train_rgb"))
    _proc(test_url, os.path.join(root, "test_rgb"))
download_rgb("/content/drive/MyDrive/data")        

0it [00:00, ?it/s]

In [ ]:
%cd /content/AdaBins

/content/AdaBins


In [ ]:
!python train.py --data_path "/content/drive/MyDrive/rgb" \
--gt_path "/content/drive/MyDrive/depth" \
--data_path_eval "/content/drive/MyDrive/rgb" \
--gt_path_eval "/content/drive/MyDrive/depth" \
--root "/content/AdaBins"

Streaming output truncated to the last 5000 lines.
Epoch: 2/25. Loop: Train:  17% 17/100 [00:29<01:35,  1.15s/it]loss tensor(0.3021, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2/25. Loop: Train:  18% 18/100 [00:30<01:34,  1.15s/it]loss tensor(0.2967, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2/25. Loop: Train:  19% 19/100 [00:31<01:32,  1.15s/it]loss tensor(0.2999, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2/25. Loop: Train:  20% 20/100 [00:32<01:31,  1.15s/it]loss tensor(0.2961, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2/25. Loop: Train:  21% 21/100 [00:33<01:31,  1.15s/it]loss tensor(0.2959, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2/25. Loop: Train:  22% 22/100 [00:34<01:29,  1.15s/it]loss tensor(0.2969, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2/25. Loop: Train:  23% 23/100 [00:35<01:28,  1.15s/it]loss tensor(0.3000, device='cuda:0', grad_fn=<AddBackward0>)
Epoch: 2/25. Loop: Train:  24% 24/100 [00:37<01:27,  1.15s/it]loss tensor(0.2952, device

In [ ]:
%cd /content/AdaBins

/content/AdaBins


In [ ]:
!python infer.py

img (480, 640, 3)
Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Loading base model ()...Done.
Removing last two layers (global_pool & classifier).
Building Encoder-Decoder model..Done.
pic (640, 480, 3)
torch.Size([1, 1, 640, 480])
(1, 640, 480, 4)
(640, 480, 4)
(640, 480, 3)
took :5.70850133895874s
<PIL.Image.Image image mode=L size=480x640 at 0x7FD3D43D0D50>
<Figure size 640x480 with 1 Axes>


In [ ]:
import shutil, sys  

shutil.move("/content/AdaBins/checkpoints/UnetAdaptiveBins.pt", "/content/drive/MyDrive") 


In [ ]:
%cd /content/AdaBins

[Errno 2] No such file or directory: '/content/AdaBins'
/content


In [ ]:
#newwwwwwwwwwwwwwwwwwwwwwwww
# -*- coding: utf-8 -*-
"""
 @Time    : 2019/3/3 19:55
 @Author  : Wang Xin
 @Email   : wangxin_buaa@163.com
"""

import torch
import torch.nn as nn
from torch.nn import BatchNorm2d
import torchvision.models.resnet

affine_par = True


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None, fist_dilation=1, multi_grid=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=dilation * multi_grid, dilation=dilation * multi_grid, bias=False)
        self.bn2 = BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=False)
        self.relu_inplace = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.dilation = dilation
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = out + residual
        out = self.relu_inplace(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, layers):
        self.inplanes = 128
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 64, stride=2)
        self.bn1 = BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = conv3x3(64, 64)
        self.bn2 = BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=False)
        self.conv3 = conv3x3(64, 128)
        self.bn3 = BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=False)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.relu = nn.ReLU(inplace=False)
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)  # change
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4, multi_grid=(1, 1, 1))

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, multi_grid=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm2d(planes * block.expansion, affine=affine_par))

        layers = []
        generate_multi_grid = lambda index, grids: grids[index % len(grids)] if isinstance(grids, tuple) else 1
        layers.append(block(self.inplanes, planes, stride, dilation=dilation, downsample=downsample,
                            multi_grid=generate_multi_grid(0, multi_grid)))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(
                block(self.inplanes, planes, dilation=dilation, multi_grid=generate_multi_grid(i, multi_grid)))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def freeze(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()


def resnet101(pretrained=True):
    resnet101 = ResNet(Bottleneck, [3, 4, 23, 3])

    if pretrained:
        # saved_state_dict = torch.load('./network/pretrained_models/resnet101-imagenet.pth')
        saved_state_dict = torch.load('/content/drive/MyDrive/resnet101-imagenet.pth')
        new_params = resnet101.state_dict().copy()
        for i in saved_state_dict:
            i_parts = i.split('.')
            if not i_parts[0] == 'fc':
                new_params['.'.join(i_parts[0:])] = saved_state_dict[i]

        resnet101.load_state_dict(new_params)
        print(
            "++++++++++++++++++++++++++++++++  Pre-trained Model Loaded        ++++++++++++++++++++++++++++++++++++++")

    return resnet101

# if __name__ == "__main__":
#     model = resnet101(pretrained=True)
#     model = model.cuda()
#     model.eval()
#     image = torch.randn(1, 3, 720, 1280)
#     image = image.cuda()
#     with torch.no_grad():
#         out0, out1 = model(image)
#     print('out0 size:', out0.size())
#     print('out1 size:', out1.size())

In [ ]:
#newwwwwwwwwwwwwwwwwwwwww
import os

import torch
import torch.nn as nn
import torchvision.models
import collections
import math

# from network.backbone import resnet101


def weights_init(modules, type='xavier'):
    m = modules
    if isinstance(m, nn.Conv2d):
        if type == 'xavier':
            torch.nn.init.xavier_normal_(m.weight)
        elif type == 'kaiming':  # msra
            torch.nn.init.kaiming_normal_(m.weight)
        else:
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))

        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.ConvTranspose2d):
        if type == 'xavier':
            torch.nn.init.xavier_normal_(m.weight)
        elif type == 'kaiming':  # msra
            torch.nn.init.kaiming_normal_(m.weight)
        else:
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))

        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Linear):
        if type == 'xavier':
            torch.nn.init.xavier_normal_(m.weight)
        elif type == 'kaiming':  # msra
            torch.nn.init.kaiming_normal_(m.weight)
        else:
            m.weight.data.fill_(1.0)

        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.Module):
        for m in modules:
            if isinstance(m, nn.Conv2d):
                if type == 'xavier':
                    torch.nn.init.xavier_normal_(m.weight)
                elif type == 'kaiming':  # msra
                    torch.nn.init.kaiming_normal_(m.weight)
                else:
                    n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                    m.weight.data.normal_(0, math.sqrt(2. / n))

                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.ConvTranspose2d):
                if type == 'xavier':
                    torch.nn.init.xavier_normal_(m.weight)
                elif type == 'kaiming':  # msra
                    torch.nn.init.kaiming_normal_(m.weight)
                else:
                    n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                    m.weight.data.normal_(0, math.sqrt(2. / n))

                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1.0)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                if type == 'xavier':
                    torch.nn.init.xavier_normal_(m.weight)
                elif type == 'kaiming':  # msra
                    torch.nn.init.kaiming_normal_(m.weight)
                else:
                    m.weight.data.fill_(1.0)

                if m.bias is not None:
                    m.bias.data.zero_()


class FullImageEncoder(nn.Module):
    def __init__(self):
        super(FullImageEncoder, self).__init__()
        self.global_pooling = nn.AvgPool2d(8, stride=8, padding=(4, 2))  # KITTI 16 16
        self.dropout = nn.Dropout2d(p=0.5)
        self.global_fc = nn.Linear(2048 * 6 * 5, 512)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(512, 512, 1)  # 1x1 卷积
        self.upsample = nn.UpsamplingBilinear2d(size=(33, 45))  # size=(33, 45))  # KITTI 49X65 NYU 33X45

        weights_init(self.modules(), 'xavier')

    def forward(self, x):

        x1 = self.global_pooling(x)
        # print('# x1 size:', x1.size())
        x2 = self.dropout(x1)
        print('# x2 size:', x2.size())
        x3 = x2.view(-1, 2048 * 6 * 5)[0] #2048 * 6 * 5
        print('# x3 size:', x3.size())
        x4 = self.relu(self.global_fc(x3))
        # print('# x4 size:', x4.size())
        x4 = x4.view(-1, 512, 1, 1)
        # print('# x4 size:', x4.size())
        x5 = self.conv1(x4)
        out = self.upsample(x5)
        print("_---------------------------------------------__")
        print("fuck shapes", x1.size(), x2.size(), x3.size(), x4.size(), x5.size(),out.size())
        print("_---------------------------------------------__")
        print('x.size', x.size(), 'out.size', out.size())
        return out


class SceneUnderstandingModule(nn.Module):
    def __init__(self):
        super(SceneUnderstandingModule, self).__init__()
        self.encoder = FullImageEncoder()
        self.aspp1 = nn.Sequential(
            nn.Conv2d(2048, 512, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 1),
            nn.ReLU(inplace=True)
        )
        self.aspp2 = nn.Sequential(
            nn.Conv2d(2048, 512, 3, padding=6, dilation=6),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 1),
            nn.ReLU(inplace=True)
        )
        self.aspp3 = nn.Sequential(
            nn.Conv2d(2048, 512, 3, padding=12, dilation=12),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 1),
            nn.ReLU(inplace=True)
        )
        self.aspp4 = nn.Sequential(
            nn.Conv2d(2048, 512, 3, padding=18, dilation=18),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 1),
            nn.ReLU(inplace=True)
        )
        self.concat_process = nn.Sequential(
            nn.Dropout2d(p=0.5),
            nn.Conv2d(512 * 4, 2048, 2),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            nn.Conv2d(2048, 136, 2),  # KITTI 142 NYU 136 In paper, K = 80 is best, so use 160 is good!
            # nn.UpsamplingBilinear2d(scale_factor=8)
            nn.UpsamplingBilinear2d(size=(257, 353))
        )

        weights_init(self.modules(), type='xavier')

    def forward(self, x):
        print('x.size', x.size())
        x1 = self.encoder(x)

        x2 = self.aspp1(x)
        print("x2",x2.shape)
        x3 = self.aspp2(x)
        print("x3",x3.shape)
        x4 = self.aspp3(x)
        print("x4",x4.shape)
        x5 = self.aspp4(x)
        print("x5",x5.shape)

        print("shapes", x1.size(), x2.size(), x3.size(), x4.size(), x5.size())
        x6 = torch.cat((x2, x3, x4, x5), dim=1)
        print("x6",x6.shape)
        # print('cat x6 size:', x6.size())
        out = self.concat_process(x6)
        print("out",out.shape)
        return out


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from DORN_pytorch.dorn.modules.backbones.resnet import ResNetBackbone
# from DORN_pytorch.dorn.modules.encoders.SceneUnderstandingModule import SceneUnderstandingModule

from models.miniViT import mViT


class UpSampleBN(nn.Module):
    def __init__(self, skip_input, output_features):
        super(UpSampleBN, self).__init__()

        self._net = nn.Sequential(nn.Conv2d(skip_input, output_features, kernel_size=3, stride=1, padding=1),
                                  nn.BatchNorm2d(output_features),
                                  nn.LeakyReLU(),
                                  nn.Conv2d(output_features, output_features, kernel_size=3, stride=1, padding=1),
                                  nn.BatchNorm2d(output_features),
                                  nn.LeakyReLU())

    def forward(self, x, concat_with):
        up_x = F.interpolate(x, size=[concat_with.size(2), concat_with.size(3)], mode='bilinear', align_corners=True)
        f = torch.cat([up_x, concat_with], dim=1)
        return self._net(f)


class DecoderBN(nn.Module):
    def __init__(self, num_features=2048, num_classes=1, bottleneck_features=2048):
        super(DecoderBN, self).__init__()
        features = int(num_features)
        self.conv2 = nn.Conv2d(bottleneck_features, features, kernel_size=1, stride=1, padding=1)

        self.up1 = UpSampleBN(skip_input=features // 1 + 112 + 64, output_features=features // 2)
        self.up2 = UpSampleBN(skip_input=features // 2 + 40 + 24, output_features=features // 4)
        self.up3 = UpSampleBN(skip_input=features // 4 + 24 + 16, output_features=features // 8)
        self.up4 = UpSampleBN(skip_input=features // 8 + 16 + 8, output_features=features // 16)

        #         self.up5 = UpSample(skip_input=features // 16 + 3, output_features=features//16)
        self.conv3 = nn.Conv2d(features // 16, num_classes, kernel_size=3, stride=1, padding=1)
        # self.act_out = nn.Softmax(dim=1) if output_activation == 'softmax' else nn.Identity()

    def forward(self, features):
        x_block0, x_block1, x_block2, x_block3, x_block4 = features[4], features[5], features[6], features[8], features[
            11]

        x_d0 = self.conv2(x_block4)

        x_d1 = self.up1(x_d0, x_block3)
        x_d2 = self.up2(x_d1, x_block2)
        x_d3 = self.up3(x_d2, x_block1)
        x_d4 = self.up4(x_d3, x_block0)
        #         x_d5 = self.up5(x_d4, features[0])
        out = self.conv3(x_d4)
        # out = self.act_out(out)
        # if with_features:
        #     return out, features[-1]
        # elif with_intermediate:
        #     return out, [x_block0, x_block1, x_block2, x_block3, x_block4, x_d1, x_d2, x_d3, x_d4]
        return out


class Encoder(nn.Module):
    def __init__(self, backend):
        super(Encoder, self).__init__()
        self.original_model = backend

    def forward(self, x):
        features = [x]
        for k, v in self.original_model._modules.items():
            if (k == 'blocks'):
                for ki, vi in v._modules.items():
                    features.append(vi(features[-1]))
            else:
                features.append(v(features[-1]))
        return features


class UnetAdaptiveBins(nn.Module):
    def __init__(self, n_bins=100, min_val=0.1, max_val=10, norm='linear'):
        super(UnetAdaptiveBins, self).__init__()
        self.num_classes = n_bins
        self.min_val = min_val
        self.max_val = max_val
        # self.encoder = Encoder(backend)
        self.adaptive_bins_layer = mViT(128, n_query_channels=128, patch_size=16,
                                        dim_out=n_bins,
                                        embedding_dim=128, norm=norm)

        self.decoder = DecoderBN(num_classes=128)
        self.conv_out = nn.Sequential(nn.Conv2d(128, n_bins, kernel_size=1, stride=1, padding=0),
                                      nn.Softmax(dim=1))
        output_size=(257, 353)
        channel=3
        self.output_size = output_size
        self.channel = channel
        self.feature_extractor = resnet101(pretrained=True)
        self.aspp_module = SceneUnderstandingModule()

    def forward(self, x, **kwargs):
        x1 = self.feature_extractor(x)
        print("follw the input x", x.size(), "x1.size", x1.size())
        print("feature_extractor",x1.shape)
        x2 = self.aspp_module(x1)
        print("x2",x2.shape)
        # unet_out = self.decoder(x2)
        # print("unet_out",unet_out.shape)
        x2 = x2.permute((0, 1, 3,2))
        print("x2",x2.shape)
        x2 = x2[:,:128,:320,:240]
        x2 = torch.reshape(x2,(2,128,320,240))
        # print("unet_out",unet_out.shape)

        bin_widths_normed, range_attention_maps = self.adaptive_bins_layer(x2)
        out = self.conv_out(range_attention_maps)

        # Post process
        # n, c, h, w = out.shape
        # hist = torch.sum(out.view(n, c, h * w), dim=2) / (h * w)  # not used for training

        bin_widths = (self.max_val - self.min_val) * bin_widths_normed  # .shape = N, dim_out
        bin_widths = nn.functional.pad(bin_widths, (1, 0), mode='constant', value=self.min_val)
        bin_edges = torch.cumsum(bin_widths, dim=1)

        centers = 0.5 * (bin_edges[:, :-1] + bin_edges[:, 1:])
        n, dout = centers.size()
        centers = centers.view(n, dout, 1, 1)

        pred = torch.sum(out * centers, dim=1, keepdim=True)

        return bin_edges, pred

    def get_1x_lr_params(self):  # lr/10 learning rate
        return self.encoder.parameters()

    def get_10x_lr_params(self):  # lr learning rate
        modules = [self.decoder, self.adaptive_bins_layer, self.conv_out]
        for m in modules:
            yield from m.parameters()



model = UnetAdaptiveBins()
x = torch.rand(2, 3, 257, 353)
bins, pred = model(x)
print(bins.shape, pred.shape)
2, 128, 240, 320 #adabin model
2, 136, 257, 353 #now 
1, 136, 257, 353 #before

++++++++++++++++++++++++++++++++  Pre-trained Model Loaded        ++++++++++++++++++++++++++++++++++++++
follw the input x torch.Size([2, 3, 257, 353]) x1.size torch.Size([2, 2048, 33, 45])
feature_extractor torch.Size([2, 2048, 33, 45])
x.size torch.Size([2, 2048, 33, 45])
# x2 size: torch.Size([2, 2048, 5, 6])
# x3 size: torch.Size([61440])
_---------------------------------------------__
fuck shapes torch.Size([2, 2048, 5, 6]) torch.Size([2, 2048, 5, 6]) torch.Size([61440]) torch.Size([1, 512, 1, 1]) torch.Size([1, 512, 1, 1]) torch.Size([1, 512, 33, 45])
_---------------------------------------------__
x.size torch.Size([2, 2048, 33, 45]) out.size torch.Size([1, 512, 33, 45])
x2 torch.Size([2, 512, 33, 45])
x3 torch.Size([2, 512, 33, 45])
x4 torch.Size([2, 512, 33, 45])
x5 torch.Size([2, 512, 33, 45])
shapes torch.Size([1, 512, 33, 45]) torch.Size([2, 512, 33, 45]) torch.Size([2, 512, 33, 45]) torch.Size([2, 512, 33, 45]) torch.Size([2, 512, 33, 45])
x6 torch.Size([2, 2048, 33, 45]

(1, 136, 257, 353)

In [ ]:
%cd /content/AdaBins

/content/AdaBins


In [ ]:
import os

import torch
import torch.nn as nn
import torchvision.models
import collections
import math

# from network.backbone import resnet101


def weights_init(modules, type='xavier'):
    m = modules
    if isinstance(m, nn.Conv2d):
        if type == 'xavier':
            torch.nn.init.xavier_normal_(m.weight)
        elif type == 'kaiming':  # msra
            torch.nn.init.kaiming_normal_(m.weight)
        else:
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))

        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.ConvTranspose2d):
        if type == 'xavier':
            torch.nn.init.xavier_normal_(m.weight)
        elif type == 'kaiming':  # msra
            torch.nn.init.kaiming_normal_(m.weight)
        else:
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))

        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Linear):
        if type == 'xavier':
            torch.nn.init.xavier_normal_(m.weight)
        elif type == 'kaiming':  # msra
            torch.nn.init.kaiming_normal_(m.weight)
        else:
            m.weight.data.fill_(1.0)

        if m.bias is not None:
            m.bias.data.zero_()
    elif isinstance(m, nn.Module):
        for m in modules:
            if isinstance(m, nn.Conv2d):
                if type == 'xavier':
                    torch.nn.init.xavier_normal_(m.weight)
                elif type == 'kaiming':  # msra
                    torch.nn.init.kaiming_normal_(m.weight)
                else:
                    n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                    m.weight.data.normal_(0, math.sqrt(2. / n))

                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.ConvTranspose2d):
                if type == 'xavier':
                    torch.nn.init.xavier_normal_(m.weight)
                elif type == 'kaiming':  # msra
                    torch.nn.init.kaiming_normal_(m.weight)
                else:
                    n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                    m.weight.data.normal_(0, math.sqrt(2. / n))

                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1.0)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                if type == 'xavier':
                    torch.nn.init.xavier_normal_(m.weight)
                elif type == 'kaiming':  # msra
                    torch.nn.init.kaiming_normal_(m.weight)
                else:
                    m.weight.data.fill_(1.0)

                if m.bias is not None:
                    m.bias.data.zero_()




class SceneUnderstandingModule(nn.Module):
    def __init__(self):
        super(SceneUnderstandingModule, self).__init__()
        self.aspp1 = nn.Sequential(
            nn.Conv2d(128, 128, 1))
        
        self.aspp2 = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=6, dilation=6))
        
        self.aspp3 = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=12, dilation=12))
        
        self.aspp4 = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=18, dilation=18))
        
        self.concat_process = nn.Sequential(
            nn.Dropout2d(p=0.5),
            nn.Conv2d(128 * 4, 128, 2),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
            nn.Conv2d(128, 128, 2),  # KITTI 142 NYU 136 In paper, K = 80 is best, so use 160 is good!
            # nn.UpsamplingBilinear2d(scale_factor=8)
            nn.UpsamplingBilinear2d(size=(240, 320))
        )

        weights_init(self.modules(), type='xavier')

    def forward(self, x):
        print('x.size', x.size())

        x2 = self.aspp1(x)
        print("x2",x2.shape)
        x3 = self.aspp2(x)
        print("x3",x3.shape)
        x4 = self.aspp3(x)
        print("x4",x4.shape)
        x5 = self.aspp4(x)
        print("x5",x5.shape)

        print("shapes", x2.size(), x3.size(), x4.size(), x5.size())
        x6 = torch.cat((x2, x3, x4, x5), dim=1)
        print("x6",x6.shape)
        # print('cat x6 size:', x6.size())
        out = self.concat_process(x6)
        print("out",out.shape)
        return out


In [ ]:
import torch
import torch.nn as nn

from models.layers import PatchTransformerEncoder,PixelWiseDotProduct


class mViT(nn.Module):
    def __init__(self, in_channels, n_query_channels=128, patch_size=16, dim_out=256,
                 embedding_dim=128, num_heads=4, norm='linear'):
        super(mViT, self).__init__()
        self.norm = norm
        self.n_query_channels = n_query_channels
        self.patch_transformer = PatchTransformerEncoder(in_channels, patch_size, embedding_dim, num_heads)
        self.dot_product_layer = PixelWiseDotProduct()

        self.conv3x3 = nn.Conv2d(in_channels, embedding_dim, kernel_size=3, stride=1, padding=1) #SceneUnderstandingModule() 
        self.convaspp = SceneUnderstandingModule() 
        self.regressor = nn.Sequential(nn.Linear(embedding_dim, 256),
                                       nn.LeakyReLU(),
                                       nn.Linear(256, 256),
                                       nn.LeakyReLU(),
                                       nn.Linear(256, dim_out))

    def forward(self, x):
        # n, c, h, w = x.size()
        x1 = torch.flatten(x)
        x1 = x1[:6082560]
        x1 = x1.view((2,2048,33,45))
        x_in = x
        tgt = self.patch_transformer(x.clone())  # .shape = S, N, E
        print("x_in",x.shape)
        print("tgt",tgt.shape)
        # x = self.conv3x3(x1)
        # x = x[:,:128,:240,:320]
        print("x_in_3*3",x.shape)
        x = self.conv3x3(x)
        print("x_out_3*3",x.shape)
        print("x_in_app",x_in.shape)
        xapp = self.convaspp(x_in)
        print("xapp",xapp.shape)
        regression_head, queries = tgt[0, ...], tgt[1:self.n_query_channels + 1, ...]
        # Change from S, N, E to N, S, E
        queries = queries.permute(1, 0, 2)
        range_attention_maps = self.dot_product_layer(x, queries)  # .shape = n, n_query_channels, h, w

        y = self.regressor(regression_head)  # .shape = N, dim_out
        if self.norm == 'linear':
            y = torch.relu(y)
            eps = 0.1
            y = y + eps
        elif self.norm == 'softmax':
            return torch.softmax(y, dim=1), range_attention_maps
        else:
            y = torch.sigmoid(y)
        y = y / y.sum(dim=1, keepdim=True)
        return y, range_attention_maps


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
# from models.miniViT import mViT


class UpSampleBN(nn.Module):
    def __init__(self, skip_input, output_features):
        super(UpSampleBN, self).__init__()

        self._net = nn.Sequential(nn.Conv2d(skip_input, output_features, kernel_size=3, stride=1, padding=1),
                                  nn.BatchNorm2d(output_features),
                                  nn.LeakyReLU(),
                                  nn.Conv2d(output_features, output_features, kernel_size=3, stride=1, padding=1),
                                  nn.BatchNorm2d(output_features),
                                  nn.LeakyReLU())

    def forward(self, x, concat_with):
        up_x = F.interpolate(x, size=[concat_with.size(2), concat_with.size(3)], mode='bilinear', align_corners=True)
        f = torch.cat([up_x, concat_with], dim=1)
        return self._net(f)


class DecoderBN(nn.Module):
    def __init__(self, num_features=2048, num_classes=1, bottleneck_features=2048):
        super(DecoderBN, self).__init__()
        features = int(num_features)

        self.conv2 = nn.Conv2d(bottleneck_features, features, kernel_size=1, stride=1, padding=1)

        self.up1 = UpSampleBN(skip_input=features // 1 + 112 + 64, output_features=features // 2)
        self.up2 = UpSampleBN(skip_input=features // 2 + 40 + 24, output_features=features // 4)
        self.up3 = UpSampleBN(skip_input=features // 4 + 24 + 16, output_features=features // 8)
        self.up4 = UpSampleBN(skip_input=features // 8 + 16 + 8, output_features=features // 16)

        #         self.up5 = UpSample(skip_input=features // 16 + 3, output_features=features//16)
        self.conv3 = nn.Conv2d(features // 16, num_classes, kernel_size=3, stride=1, padding=1)
        # self.act_out = nn.Softmax(dim=1) if output_activation == 'softmax' else nn.Identity()

    def forward(self, features):
        x_block0, x_block1, x_block2, x_block3, x_block4 = features[4], features[5], features[6], features[8], features[
            11]

        x_d0 = self.conv2(x_block4)

        x_d1 = self.up1(x_d0, x_block3)
        x_d2 = self.up2(x_d1, x_block2)
        x_d3 = self.up3(x_d2, x_block1)
        x_d4 = self.up4(x_d3, x_block0)
        #         x_d5 = self.up5(x_d4, features[0])
        out = self.conv3(x_d4)
        # out = self.act_out(out)
        # if with_features:
        #     return out, features[-1]
        # elif with_intermediate:
        #     return out, [x_block0, x_block1, x_block2, x_block3, x_block4, x_d1, x_d2, x_d3, x_d4]
        return out


class Encoder(nn.Module):
    def __init__(self, backend):
        super(Encoder, self).__init__()
        self.original_model = backend

    def forward(self, x):
        features = [x]
        for k, v in self.original_model._modules.items():
            if (k == 'blocks'):
                for ki, vi in v._modules.items():
                    features.append(vi(features[-1]))
            else:
                features.append(v(features[-1]))
        return features


class UnetAdaptiveBins(nn.Module):
    def __init__(self, backend, n_bins=100, min_val=0.1, max_val=10, norm='linear'):
        super(UnetAdaptiveBins, self).__init__()
        self.num_classes = n_bins
        self.min_val = min_val
        self.max_val = max_val
        self.encoder = Encoder(backend)
        self.adaptive_bins_layer = mViT(128, n_query_channels=128, patch_size=16,
                                        dim_out=n_bins,
                                        embedding_dim=128, norm=norm)

        self.decoder = DecoderBN(num_classes=128)
        self.conv_out = nn.Sequential(nn.Conv2d(128, n_bins, kernel_size=1, stride=1, padding=0),
                                      nn.Softmax(dim=1))

    def forward(self, x, **kwargs):
        print("in",x.shape)
        x1=self.encoder(x)
        unet_out = self.decoder(x1, **kwargs)
        print("unet_out",unet_out.shape,type(unet_out))
        
        bin_widths_normed, range_attention_maps = self.adaptive_bins_layer(unet_out)
        out = self.conv_out(range_attention_maps)

        # Post process
        # n, c, h, w = out.shape
        # hist = torch.sum(out.view(n, c, h * w), dim=2) / (h * w)  # not used for training

        bin_widths = (self.max_val - self.min_val) * bin_widths_normed  # .shape = N, dim_out
        bin_widths = nn.functional.pad(bin_widths, (1, 0), mode='constant', value=self.min_val)
        bin_edges = torch.cumsum(bin_widths, dim=1)

        centers = 0.5 * (bin_edges[:, :-1] + bin_edges[:, 1:])
        n, dout = centers.size()
        centers = centers.view(n, dout, 1, 1)

        pred = torch.sum(out * centers, dim=1, keepdim=True)

        return bin_edges, pred

    def get_1x_lr_params(self):  # lr/10 learning rate
        return self.encoder.parameters()

    def get_10x_lr_params(self):  # lr learning rate
        modules = [self.decoder, self.adaptive_bins_layer, self.conv_out]
        for m in modules:
            yield from m.parameters()

    @classmethod
    def build(cls, n_bins, **kwargs):
        basemodel_name = 'tf_efficientnet_b5_ap'

        print('Loading base model ()...'.format(basemodel_name), end='')
        basemodel = torch.hub.load('rwightman/gen-efficientnet-pytorch', basemodel_name, pretrained=True)
        print('Done.')

        # Remove last layer
        print('Removing last two layers (global_pool & classifier).')
        basemodel.global_pool = nn.Identity()
        basemodel.classifier = nn.Identity()

        # Building Encoder-Decoder model
        print('Building Encoder-Decoder model..', end='')
        m = cls(basemodel, n_bins=n_bins, **kwargs)
        print('Done.')
        return m

model = UnetAdaptiveBins.build(100)
x = torch.rand(2, 3, 480, 640)
bins, pred = model(x)
print(bins.shape, pred.shape)        

Loading base model ()...

Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Done.
Removing last two layers (global_pool & classifier).
Building Encoder-Decoder model..Done.
in torch.Size([2, 3, 480, 640])
unet_out torch.Size([2, 128, 240, 320]) <class 'torch.Tensor'>
x_in torch.Size([2, 128, 240, 320])
tgt torch.Size([300, 2, 128])
x_in_3*3 torch.Size([2, 128, 240, 320])
x_out_3*3 torch.Size([2, 128, 240, 320])
x_in_app torch.Size([2, 128, 240, 320])
x.size torch.Size([2, 128, 240, 320])
x2 torch.Size([2, 128, 240, 320])
x3 torch.Size([2, 128, 240, 320])
x4 torch.Size([2, 128, 240, 320])
x5 torch.Size([2, 128, 240, 320])
shapes torch.Size([2, 128, 240, 320]) torch.Size([2, 128, 240, 320]) torch.Size([2, 128, 240, 320]) torch.Size([2, 128, 240, 320])
x6 torch.Size([2, 512, 240, 320])
out torch.Size([2, 128, 240, 320])
xapp torch.Size([2, 128, 240, 320])
torch.Size([2, 101]) torch.Size([2, 1, 240, 320])


In [ ]:
x = torch.rand(2, 128, 240, 320)
x= torch.flatten(x)
print("x",x.shape)
x = x[:6082560]
x= x.view((2,2048,33,45))

x torch.Size([19660800])


In [ ]:
!git clone https://github.com/DaHaiHuha/DORN_pytorch.git

Cloning into 'DORN_pytorch'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 43 (delta 0), reused 6 (delta 0), pack-reused 37
Unpacking objects: 100% (43/43), done.


In [ ]:
!pip install trimesh
!pip install pyrender

In [ ]:
!pip install open3d

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import open3d as o3d
import numpy as np 

with h5py.File("/content/AdaBins/test_imgs/color_normal_mean.hdf5", "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    # a_group_key = list(f.keys())[0]
    # # Get the data
    # data = list(f[a_group_key])
    # print(f["voxelgrid"][0])
    voxelarray = np.array(f["normal"][0],dtype=np.float32)
    print(voxelarray.shape)

o3d.visualization.draw_geometries([voxelarray])

# prepare some coordinates
# x, y, z = np.indices((4, 4, 8))

# # draw cuboids in the top left and bottom right corners, and a link between
# # them
# cube1 = (x < 3) & (y < 3) & (z < 3)
# cube2 = (x >= 5) & (y >= 5) & (z >= 5)
# link = abs(x - y) + abs(y - z) + abs(z - x) <= 2

# # combine the objects into a single boolean array
# voxelarray = cube1 | cube2 | link

# # set the colors of each object
# colors = np.empty(voxelarray.shape, dtype=object)
# colors[link] = 'red'
# colors[cube1] = 'blue'
# colors[cube2] = 'green'

# # and plot everything
# ax = plt.figure().add_subplot(projection='3d')
# ax.voxels(voxelarray, facecolors=colors, edgecolor='k')

# plt.show()